## GAN : Generative Adversarial Networks

##### Practice thesis code

In [1]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

#### 생성자(Generator) 및 판별자(Discriminator)모델 정의

In [2]:
latent_dim=100

# 생성자 클래스 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # 하나의 블록(block)정의
        def block(input_dim, output_dim, normalize=True):
            layers=[nn.Linear(input_dim, output_dim)]
            if normalize:
                # 배치 정규화(batch normaliztion) 수행(차원 동일)
                layers.append(nn.BatchNorm1d(output_dim, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # 생성자 모델은 연속적인 여러 개의 블록을 가짐
        self.model=nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img


In [17]:
# 판별자 클래스 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model=nn.Sequential(
            nn.Linear(1*28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    # 이미지에 대한 판별 결과를 반환
    def forward(self, img):
        flattened = img.view(img.size(0), -1)
        output = self.model(flattened)

        return output

### 학습 데이터셋 불러오기

##### * MNIST DATA SET

In [18]:
transforms_train=transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

train_dataset=datasets.MNIST(root="./dataset", train=True, download=True, transform=transforms_train)
dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)

### 모델 학습 및 샘플링

##### * 학습을 위해 생성자와 판별자 모델 초기화
##### * 적절한 하이퍼 파라미터 설정

In [19]:
# 생성자와 판별자 초기화
generator = Generator()
discriminator = Discriminator()

generator.cuda()
discriminator.cuda()

# 손실함수 (loss function)
adversarial_loss=nn.BCELoss()
adversarial_loss.cuda()

# 학습률(learning rate)설정
lr=0.0002

# 생성자와 판별자를 위한 최적화 함수
optimizer_G=torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D=torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

In [20]:
import time

n_epochs=200 # 학습의 횟수(epoch) 설정
sample_interval=2000 # 몇 번의 배치(batch)마다 결과를 출력할 것인지 결과를 설정
start_time=time.time()

for epoch in range(n_epochs):
  for i, (imgs, _) in enumerate(dataloader):

    # 진짜(real) 이미지와 가짜(fake) 이미지에 대한 정답 레이블 생성
    real = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(1.0) # Real img : 1
    fake = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(0.0) # Fake img : 0

    real_imgs=imgs.cuda()

    """생성자(generator)를 학습합니다."""
    optimizer_G.zero_grad()

    # 랜덤 노이즈(noise) 샘플링
    z = torch.normal(mean=0, std=1, size=(imgs.shape[0], latent_dim)).cuda()

    # 이미지 생성
    generated_imgs=generator(z)

    # 생성자(generator)의 손실(loss)값 계산
    g_loss = adversarial_loss(discriminator(generated_imgs), real)

    # 생성자(generator) 업데이트
    g_loss.backward()
    optimizer_G.step()

    """판별자(discriminator)를 학습합니다."""
    optimizer_D.zero_grad()

    # 판별자(discriminator)의 손실(loss) 값 계산
    real_loss = adversarial_loss(discriminator(real_imgs), real)
    fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)
    d_loss = (real_loss + fake_loss) / 2

    # 판별자(diswcriminator) 업데이트
    d_loss.backward()
    optimizer_D.step()

    done=epoch+len(dataloader)+i
    if done % sample_interval == 0:
      # 생성된 이미지 중에서 25개만 선택하여 5*5 격자 이미지에 출력
      save_image(generated_imgs.data[:25], f"{done}.png", nrow=5, normalize=True)

    # 하나의 epoch이 끝낼 때마다 로그(log) 출력
    print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[Epoch 189/200] [D loss: 0.260931] [G loss: 3.872864] [Elapsed time: 3178.14s]
[Epoch 189/200] [D loss: 0.174466] [G loss: 2.216438] [Elapsed time: 3178.16s]
[Epoch 189/200] [D loss: 0.213014] [G loss: 3.854607] [Elapsed time: 3178.21s]
[Epoch 189/200] [D loss: 0.193818] [G loss: 2.252989] [Elapsed time: 3178.23s]
[Epoch 189/200] [D loss: 0.207859] [G loss: 3.283633] [Elapsed time: 3178.29s]
[Epoch 189/200] [D loss: 0.236057] [G loss: 2.160597] [Elapsed time: 3178.30s]
[Epoch 189/200] [D loss: 0.212405] [G loss: 4.241878] [Elapsed time: 3178.35s]
[Epoch 189/200] [D loss: 0.148611] [G loss: 3.000369] [Elapsed time: 3178.37s]
[Epoch 189/200] [D loss: 0.195950] [G loss: 2.481752] [Elapsed time: 3178.42s]
[Epoch 189/200] [D loss: 0.150787] [G loss: 2.882095] [Elapsed time: 3178.44s]
[Epoch 189/200] [D loss: 0.186671] [G loss: 2.633947] [Elapsed time: 3178.48s]
[Epoch 189/200] [D loss: 0.143483] [G loss: 3.028059] [Elapsed time: 3178.50s]
[Epoch 189/200] 

In [22]:
from IPython.display import Image
